# 2D Phase field solver 
-by Taradutt  
#### Allen-Cahn equation
The functional is,
$$ I = \int \big(\frac{K}{2} (\nabla c)^2 + f(c) \big) dx $$
The free energy in terms of the order parameter $c$ is given by the double well potential
$$ f(c) = Wc^2(1-c)^2 $$
<br>
The Allen-Cahn evolution equation is,
$$\frac{\partial c}{\partial t} = - M \Big[\frac{\partial f}{\partial c} - K \nabla^2 c\Big] $$

where,
$$ \frac{\partial f}{\partial c} = 2Wc(1-c)(1-2c) $$
  
#### Cahn-Hilliard equation
For the functional
$$ I = \int \big(\frac{K}{2} (\nabla c)^2 + f(c) \big) dx $$
the evolution equation is
$$ \frac{\partial c}{\partial t} = \nabla\cdot(M\nabla \mu) = M\nabla^2\mu \quad(\text{when M is constant)},$$ 
where $\mu$ is,
$$\mu = \frac{\partial f}{\partial c} - K \nabla^2 c $$

Similar to the Allen-Cahn equation, $f(c)$ is the double well potential,
$$ f(c) = Wc^2(1-c)^2 $$
<br>
and $$ \frac{\partial f}{\partial c} = 2Wc(1-c)(1-2c) $$
### Objective:  
Using both Cahn-Hilliard and Allen-Cahn equation, find the composition profile for initial composition of :  
$$ c_0, \frac{c_0+c_1}{2}and \frac{c_0+c_2}{2}$$ where $$c_1 and c_2$$ are points of inflection of F vs c curve.

### Solution:
Double differentiating F wrt c, yields:
$$ \frac{\partial^2 f}{\partial c^2} = 2Wc(6c^2-6c+1) $$  
On solving the quadratic we get the inflection points as  
$$c_1,c_2 = 0.5 \pm \frac{1}{2\sqrt{3}}$$
On plugging the values, we see that we need to solve for initial composition of 0.5, 0.36 and 0.64. The following code achieves that.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm,animation
from IPython.display import HTML

In [2]:
#setting the parameters
dx = 1.0
dy = 1.0
dt = 0.005
M = 1.0
K = 2.0
W = 9.0
mesh_x,mesh_y = 50, 50

In [3]:
#initializes the domain with given initial average composition
def initialize_random(c_avg,c_amp): 
    c = np.zeros((mesh_x, mesh_y))
    for i in range(mesh_x):
        for j in range(mesh_y):
            c[i,j] = c_avg + c_amp*np.random.uniform(-1,1)#adds random noise of amplitude
    #next we reset the average back to c_avg
    num = mesh_x*mesh_y
    c_subtract = np.sum(c)/num - c_avg 
    for i in range(mesh_x):
        for j in range(mesh_y):
            c[i,j] = c[i,j] - c_subtract
    return c


def d_free_energy(c):
    return 2.0*W*c*(1.0-c)*(1.0-2*c)
#2D laplacian
def laplacian(c):
    laplacian = np.zeros((mesh_x, mesh_y))
    
    for i in range(1, mesh_x-1):
        for j in range(1, mesh_y-1):
            laplacian[i,j] = ( c[i+1,j] - 2.0*c[i,j] + c[i-1,j] )/(dx**2) + ( c[i,j+1] - 2.0*c[i,j] + c[i,j-1] )/(dy**2)
    
    return laplacian
#applies periodic boundary conditions
def apply_BC(c):
    c[0,:] = c[mesh_x-4,:]
    c[:,0] = c[:,mesh_y-4]
    
    c[1,:] = c[mesh_x-3,:]
    c[:,1] = c[:,mesh_y-3]
    
    c[mesh_x-2,:] = c[2,:]
    c[:,mesh_y-2] = c[:,2]
    
    c[mesh_x-1,:] = c[3,:]
    c[:,mesh_y-1] = c[:,3]
    return c
#allen-cahn solver
def AC(c_avg, c_amp, total_t, te):#inputs: average initial composition, fluctuation amplitute, total timesteps, and interval at which we output results
    c = initialize_random(c_avg, c_amp)
    dc_dt = np.zeros((mesh_x, mesh_y))
    img_data = []
    fig = plt.figure(figsize = (5,5))
    for t in range(total_t):
        dc_dt = -M*( d_free_energy(c) - K*laplacian(c) )
        c = c + dc_dt*dt
        if t%te==0:
            
            s=plt.imshow(c, cmap = 'Spectral', vmin = 0, vmax = 1)
            plt.title('c_avg = '+str(c_avg)+',\n by Allen-Cahn')
            img_data.append([s])
    plt.colorbar()
    ani = animation.ArtistAnimation(fig, img_data, interval = 400)
    plt.close()
    display(HTML(ani.to_html5_video()))
    ani.save('c_avg = '+str(c_avg)+'_by_AC.mp4')    
#cahn-hilliard solver
def CH(c_avg, c_amp, total_t, te):
    c = initialize_random(c_avg, c_amp)
    img_data = []
    fig = plt.figure(figsize = (5,5))
    for t in range(total_t):
        mu = d_free_energy(c) - K*laplacian(c)
        dc_dt = M*laplacian(mu)
        c = c + dc_dt*dt
        c = apply_BC(c)
        if t%te==0:
            s=plt.imshow(c, cmap = 'Spectral', vmin = 0, vmax = 1)
            plt.title('c_avg = '+str(c_avg)+',\n by Cahn-Hilliard')
            img_data.append([s])
    plt.colorbar()
    ani = animation.ArtistAnimation(fig, img_data, interval = 400)
    plt.close()
    display(HTML(ani.to_html5_video()))
    ani.save('c_avg = '+str(c_avg)+'_by_CH.mp4')    

In [4]:
AC(0.5,0.05,2501,100)# till 2501 steps, every 100th step

In [5]:
AC(0.36,0.05,101,10)#rapid convergence happens, so we see every 10 steps till 100

In [6]:
AC(0.64,0.05,101,10)#rapid convergence happens, so we see every 10 steps till 100

### Observations from the above simulations which used Allen-Cahn equation:
- For the case where starting composition is 0.5, we get phase separation. Interface concentration remains close to 0.5.  
- When we deviate from 0.5, we dont get phase separation. Rather a single phase results from rapid convergence of the concentration to either 0 or 1, depending on the initial average composition.
- This happens because, when we choose initial average concentration as 0.5, which is a local maxima of the F vs c curve, there is no barrier to fall down to the minimas at 0,1. So due to the initial fluctuations being random around the peak value of 0.5, half the regions roll down the potential hill towards the minima at 0 and the other half goes towards 1. This is followed by coarsening.
- However when we deviate from 0.5, the system proceeds towards the nearest minima of the double well function(As going to the faraway minima would require crossing the barrier at 0.5). So when we choose c_avg > 0.5, for eg. as in the case of c_avg = 0.64, the system goes to nearest minima of c = 1, whereas it proceeds towards the nearest minima of c = 0, when we choose initial average concentration as 0.36.

In [7]:
CH(0.5,0.05,6001,400)

In [8]:
CH(0.36,0.05,6001,400)

In [9]:
CH(0.64,0.05,6001,400)

### Observations from the above simulations which used Cahn-Hilliard equation:

- For the case where initial composition is 0.5, we get a lamellar microstructure resulting from spinodal decomposition. This microstructural pattern is different from what we obtained using Allen-Cahn equation, where it was mostly random.
- In this case, when we deviate from initial average concentration of 0.5, we get precipitation of one phase in matrix of another. The matrix is formed by the phase that has concentration of the minima nearer to the average initial concentration. And the precipitate is formed by the phase that has concentration given by the minima far away from the average initial concentration. For e.g., when the initial average concentration is 0.64, the nearer minima of F vs c curve is 1, so the matrix is formed by phase with concentration equal to 1.0, while the precipitate is formed by phase with concentration of 0. Similarly when initial concentration is 0.36, the matrix concentration is 0 while the precipitate concentration is 1.
- Here in all the cases the average concentration looks to remain the same as opposed to the Allen-Cahn case where it went to either 0 or 1 when we deviated from initial composition of 0.5 
- We get different results from Cahn-Hilliard and Allen-Cahn for the same initial average compositions because although they look similar at first glance, the dynamics governing both models are quite different. Also Allen-Cahn doesn't conserve the order parameter.
